In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet

# **KP 데이터 전처리**

In [ ]:
kp_2020 = pd.read_csv('/content/drive/MyDrive/cau_file/research/national_security/KP2020.csv', encoding='cp949')
kp_2021 = pd.read_csv('/content/drive/MyDrive/cau_file/research/national_security/KP2021.csv', encoding='cp949')

In [ ]:
kp_2020.tail(5)

In [ ]:
kp_2021.tail(5)

In [ ]:
# DataFrame concatenate

total_kp = pd.concat([kp_2020, kp_2021])

In [ ]:
total_kp.tail(5)

,RECV_DEPT_NM,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
2594055,대전청,23/01/12 11:23:09.000000000,13,10,104,1.0,대전광역시 유성구 봉명동 (행정:원신흥동 ) 1016-7,127.344600,36.352272,NaN
2594056,충남청,23/01/16 11:59:47.000000000,19,10,307,2.0,충청남도 서산시 음암면 부장리(음암면 ) 560-4,126.514047,36.785626,NaN
2594057,충남청,23/01/16 11:58:54.000000000,19,10,601,3.0,NaN,NaN,NaN,NaN
2594058,충남청,23/01/14 01:00:47.000000000,19,10,601,3.0,NaN,NaN,NaN,NaN
2594059,대전청,23/01/14 01:22:31.000000000,13,10,606,1.0,NaN,127.404663,36.341685,NaN


In [ ]:
# total KP's information

total_kp.info()
print()
print(f'rows: {total_kp.shape[0]}, columns: {total_kp.shape[1]}')

In [ ]:
npa_cl = total_kp['NPA_CL'].unique()
npa_cl.sort()
print(npa_cl)

[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 31]


In [ ]:
from collections import OrderedDict

npa_cl_dict = OrderedDict()
npa_cl_dict = {
    '8' : '서울청',
    '9' : '부산청',
    '10' : '대구청',
    '11' : '인천청',
    '12' : '광주청',
    '13' : '대전청',
    '14' : '울산청',
    '15' : '경기남부청',
    '16' : '경기북부청',
    '17' : '강원청',
    '18' : '충북청',
    '19' : '충남청',
    '20' : '전북청',
    '21' : '전남청',
    '22' : '경북청',
    '23' : '경남청',
    '24' : '제주청',
    '31' : '세종청', 
}

In [ ]:
# 필요없는 컬럼 삭제

new_kp = total_kp.drop(['RPTER_SEX', 'HPPN_PNU_ADDR', 'HPPN_X', 'HPPN_Y', 'SME_EVT_YN', 'EVT_CL_CD', 'EVT_STAT_CD', 'RECV_DEPT_NM'], axis=1)

In [ ]:
new_kp.tail(5)

,RECV_CPLT_DM,NPA_CL
2594055,23/01/12 11:23:09.000000000,13
2594056,23/01/16 11:59:47.000000000,19
2594057,23/01/16 11:58:54.000000000,19
2594058,23/01/14 01:00:47.000000000,19
2594059,23/01/14 01:22:31.000000000,13


In [ ]:
new_kp['datetime'] = pd.to_datetime(new_kp['RECV_CPLT_DM'], format='%y/%m/%d %H:%M:%S.%f') 

In [ ]:
new_kp.tail()

,RECV_CPLT_DM,NPA_CL,datetime
2594055,23/01/12 11:23:09.000000000,13,2023-01-12 11:23:09
2594056,23/01/16 11:59:47.000000000,19,2023-01-16 11:59:47
2594057,23/01/16 11:58:54.000000000,19,2023-01-16 11:58:54
2594058,23/01/14 01:00:47.000000000,19,2023-01-14 01:00:47
2594059,23/01/14 01:22:31.000000000,13,2023-01-14 01:22:31


In [ ]:
new_kp['datetime'] = new_kp['datetime'].dt.strftime('%y/%m/%d')

In [ ]:
new_kp['datetime'] = pd.to_datetime(new_kp['datetime'], format='%y/%m/%d')

In [ ]:
new_kp = new_kp.drop(['RECV_CPLT_DM'], axis=1)
# new_kp = new_kp.set_index('datetime')

In [ ]:
new_kp.tail()

,NPA_CL,datetime
2594055,13,2023-01-12
2594056,19,2023-01-16
2594057,19,2023-01-16
2594058,19,2023-01-14
2594059,13,2023-01-14


In [ ]:
new_kp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2671137 entries, 0 to 2594059
Data columns (total 2 columns):
 #   Column    Dtype         
---  ------    -----         
 0   NPA_CL    int64         
 1   datetime  datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 125.6 MB


In [ ]:
new_kp.value_counts()

NPA_CL  datetime  
19      2022-06-07    3558
        2022-10-15    2982
        2021-10-09    2923
        2022-06-25    2829
        2022-07-15    2781
                      ... 
12      2021-08-25       1
21      2022-11-17       1
12      2021-08-31       1
21      2022-11-21       1
17      2022-02-11       1
Length: 9638, dtype: int64

In [ ]:
seoul_kp = new_kp[new_kp['NPA_CL'] == 8]
busan_kp = new_kp[new_kp['NPA_CL'] == 9]
daegu_kp = new_kp[new_kp['NPA_CL'] == 10]
incheon_kp = new_kp[new_kp['NPA_CL'] == 11]
gwangju_kp = new_kp[new_kp['NPA_CL'] == 12]
daejeon_kp = new_kp[new_kp['NPA_CL'] == 13]
ulsan_kp = new_kp[new_kp['NPA_CL'] == 14]
g_south_kp = new_kp[new_kp['NPA_CL']==15]
g_north_kp = new_kp[new_kp['NPA_CL'] == 16]
gangwon_kp = new_kp[new_kp['NPA_CL'] == 17]
chungbuk_kp = new_kp[new_kp['NPA_CL'] == 18]
chungnam_kp = new_kp[new_kp['NPA_CL'] == 19]
jeonbuk_kp = new_kp[new_kp['NPA_CL'] == 20]
jeonnam_kp = new_kp[new_kp['NPA_CL'] == 21]
gyeongbuk_kp = new_kp[new_kp['NPA_CL'] == 22]
gyeongnam_kp = new_kp[new_kp['NPA_CL'] == 23]
jeju_kp = new_kp[new_kp['NPA_CL'] == 24]
saejong_kp = new_kp[new_kp['NPA_CL'] == 31]

In [ ]:
seoul_kp['datetime'].value_counts()

2022-10-30    148
2022-05-20     19
2022-05-30     18
2022-05-29     18
2022-10-25     18
             ... 
2022-07-15      1
2021-12-06      1
2020-12-27      1
2021-02-17      1
2021-02-23      1
Name: datetime, Length: 777, dtype: int64

# **단순 날짜별 신고접수 DataFrame**

In [ ]:
# 단순 날짜별 신고접수 건 수

count_kp = new_kp.drop(['NPA_CL'], axis=1)

In [ ]:
count_kp = count_kp.value_counts().rename_axis('datetime').reset_index(name='case_counts')

In [ ]:
count_kp

,datetime,case_counts
0,2022-06-07,5187
1,2022-10-15,5119
2,2021-10-09,5009
3,2022-10-03,4978
4,2022-10-01,4809
...,...,...
773,2022-02-06,1996
774,2020-12-01,1947
775,2020-12-03,1917
776,2020-12-02,1777


In [ ]:
count_kp = count_kp.sort_values(by='datetime')
count_kp = count_kp.set_index('datetime')
count_kp

,case_counts
datetime,
2020-12-01,1947
2020-12-02,1777
2020-12-03,1917
2020-12-04,2074
2020-12-05,2123
...,...
2023-01-14,3663
2023-01-15,3146
2023-01-16,3452


In [ ]:
%cd /content/drive/MyDrive/cau_file/research/national_security

/content/drive/MyDrive/cau_file/research/national_security


In [ ]:
count_kp.to_csv('CaseCount_KP.csv')